# Dependency parsing baseline


Dependency parsing is the task of mapping a sentence to a formal representation of its syntactic structure in the form of a dependency tree, which consists of directed arcs between individual words (tokens). Here we will implement a dependency parser baseline based on the arc-standard algorithm and the fixed-window model that we implemented in Lab L3.

## Imports

In [1]:
from batchify import *
from create_vocab import *
from data_handling import *
from parser import *
from projectivize import *
from uas import *
from window_models import *
from taggers import *

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
device

device(type='cpu')

## The data set

In [4]:
train_data = Dataset('./data/en_gum-ud-train-projectivized.conllu')
dev_data = Dataset('./data/en_gum-ud-dev-projectivized.conllu')
test_data = Dataset('./data/en_gum-ud-test-projectivized.conllu')

## Train tagger

In [5]:
import torch.optim as optim
import torch
import torch.nn.functional as F

def train_fixed_window(train_data, n_epochs=2, batch_size=100, lr=1e-2):
    vocab_words, vocab_tags = make_vocabs(train_data)
    tagger = FixedWindowTagger(vocab_words, vocab_tags, len(vocab_tags))
    
    optimizer = optim.Adam(tagger.model.parameters(), lr=lr)
    for i in range(n_epochs):
        total_loss = 0
        batch_nr = 0
        for x, y in training_examples_tagger(vocab_words, vocab_tags, train_data, tagger):
            batch_nr += 1
            
            optimizer.zero_grad()
            y_pred = tagger.model.forward(x)
            
            loss = F.cross_entropy(y_pred, y)
            loss.backward()
            total_loss += loss.item()
            optimizer.step()
            if batch_nr % 100 == 1:
                print(total_loss/batch_nr)
                #pass
    return tagger


In [6]:
tagger = train_fixed_window(train_data)

2.988539218902588
1.0766507263230805
0.7761470550921425
0.6433389592556858
0.5670698325049848
0.5218889229222686
0.4924586370165852
0.4737867672798807
0.4521804275491116
0.41529327630996704
0.1878917758712674
0.14903390525020102
0.13142143503803005
0.12925835619994122
0.1302114050814923
0.12821916359462798
0.12605362650360408
0.1250453132644212


In [7]:
accuracy(tagger, dev_data)

0.8818215114149677

## Train parser

In [8]:
import torch.optim as optim
import torch
import torch.nn.functional as F
def train_fixed_parser(train_data, n_epochs=2, batch_size=100, lr=1e-2):
    vocab_words, vocab_tags = make_vocabs(train_data)
    parser = FixedWindowParser(vocab_words, vocab_tags)
    
    optimizer = optim.Adam(parser.model.parameters(), lr=lr)
    for i in range(n_epochs):
        total_loss = 0
        batch_nr = 0
        for x, y in training_examples_parser(vocab_words, vocab_tags, train_data, parser):
            batch_nr += 1
            
            optimizer.zero_grad()
            y_pred = parser.model.forward(x)
            
            loss = F.cross_entropy(y_pred, y)
            loss.backward()
            total_loss += loss.item()
            optimizer.step()
            if batch_nr % 100 == 1:
                print(total_loss/batch_nr)
                
    return parser

In [9]:
parser = train_fixed_parser(train_data)

1.3709524869918823
0.67494798414778
0.5991728283576111
0.5571447691549098
0.5313198175513536
0.5055585305788322
0.488999368991709
0.4782408505423093
0.46919370694702184
0.4620046924373815
0.4573909454531484
0.4526904279215568
0.44704477655872715
0.4444693769747069
0.4410749453676334
0.4351727965670693
0.42888487999715486
0.423105035658306
0.41803419975556644
0.41540059313640537
0.4107631054231669
0.4052849263271963
0.4013678312812784
0.4001972087291027
0.40037046614468025
0.39861359635367244
0.3970938696497704
0.3954218155416525
0.39414156328451844
0.3923030519901021
0.39210728127279704
0.39249709338041056
0.3921419018958089
0.39205022081868174
0.3923011830821962
0.3917929880103957
0.3921838573412129
0.3929796988299024
0.3916039378581957
0.3917746496145938
0.39148327652099607
0.3905888292379814
0.38949928266080297
0.38854517761450735
0.38832963411492044
0.3867926941412776
0.38577666895784285
0.44775471091270447
0.35572939476754406
0.3588000005157433
0.35731272523189306
0.35379023993030

In [10]:
uas(parser, dev_data)

/Users/anton/git/nlp-project/parser.py:143: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  pred = torch.nn.functional.log_softmax(pred)


0.7486216181561739

In [11]:
for i, data in enumerate(dev_data):
    if i == 3:
        uas(parser, [data])
        print(data)
        break

[('<root>', '<root>', 0), ('However', 'ADV', 6), (',', 'PUNCT', 1), ('it', 'PRON', 6), ('is', 'AUX', 6), ('not', 'PART', 6), ('enough', 'ADJ', 0), ('to', 'PART', 9), ('have', 'AUX', 9), ('attained', 'VERB', 6), ('such', 'ADJ', 12), ('native-like', 'ADJ', 12), ('levels', 'NOUN', 9), ('.', 'PUNCT', 6)]


## Testing parser with predicted tags 

In [12]:
def calc_uas_with_tagger_preds(tagger, parser, data):
    correct = 0
    total = 0
    
    new_data = []
    for sent in data:
        pred_tags = tagger.predict(sent)
    
        # Replace gold tags with predicted
        for i , (_, tag) in enumerate(pred_tags):
            sent[i] = (sent[i][0], tag, sent[i][2])
        new_data.append(sent)
        
    return uas(parser, new_data)

In [13]:
calc_uas_with_tagger_preds(tagger, parser, dev_data)

0.6929093473522246